<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 220px; height: 150px; vertical-align: middle;">
            <img src="../assets/aaa.png" width="220" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Autonomous Traders</h2>
            <span style="color:#ff7800;">An equity trading simulation to illustrate autonomous agents powered by tools and resources from MCP servers.
            </span>
        </td>
    </tr>
</table>

### Week 6 Day 4

And now - introducing the Capstone project:


# Autonomous Traders

An equity trading simulation, with 4 Traders and a Researcher, powered by a slew of MCP servers with tools & resources:

1. Our home-made Accounts MCP server (written by our engineering team!)
2. Fetch (get webpage via a local headless browser)
3. Memory
4. Brave Search
5. Financial data

And a resource to read information about the trader's account, and their investment strategy.

The goal of today's lab is to make a new python module, `traders.py` that will manage a single trader on our trading floor.

We will experiment and explore in the lab, and then migrate to a python module when we're ready.


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">One more time --</h2>
            <span style="color:#ff7800;">Please **DO NOT** use this for actual trading decisions!!
            </span>
        </td>
    </tr>
</table>

In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

### Let's start by gathering the MCP params for our trader

In [2]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


In [3]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### And now for our researcher

In [4]:
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

### Now create the MCPServerStdio for each

In [5]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research

And turn it into a tool - remember how this works for OpenAI Agents SDK, and the difference with handoffs?

In [6]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [7]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [8]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))



Here are the latest news highlights on Amazon:

1. Amazon Prime Day 2025 is scheduled for July 8-11, with early access deals available now for Prime members. The event will last 4 days this year, doubling in length from previous years.

2. Amazon has issued a scam warning for consumers ahead of the upcoming July 4th sales to be cautious of fraudulent activities.

3. Amazon announced a major expansion of its Prime delivery network, extending same-day and next-day delivery to over 4,000 rural areas by the end of 2025.

4. CEO Andy Jassy revealed that Amazon plans to reduce its total corporate workforce in the next few years due to increased AI automation replacing some jobs.

5. Amazon is investing $10 billion in new data centers in North Carolina to expand its AI computing footprint, which is expected to create 500 jobs.

6. Nike will raise prices and resume selling its products on Amazon after a more than five-year break.

7. On a different note, Jeff Bezos, Amazon's founder, recently married Lauren Sanchez in a lavish, star-studded wedding held in Venice, Italy, which attracted significant media and protest attention.

If you need any more detailed info or have other questions, just let me know!

### Look at the trace

https://platform.openai.com/traces

In [9]:
ed_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Ed").reset(ed_initial_strategy)

display(Markdown(await read_accounts_resource("Ed")))
display(Markdown(await read_strategy_resource("Ed")))

{"name": "ed", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-07-04 12:57:37", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

### And now - to create our Trader Agent

In [10]:
agent_name = "Ed"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [11]:
print(instructions)


You are a trader that manages a portfolio of shares. Your name is Ed and your account is under your name, Ed.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "ed", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-07-04 12:57:37", 10000.0], ["2025-07-04 12:57:38", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please

### And to run our Trader

In [12]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

### Summary of Actions Taken

1. **Market Research**:
   - The stock market is buoyed by optimism surrounding trade talks and a potential Federal Reserve rate cut. The tech sector is showing strong performance.

2. **Stock Prices for Key Holdings**:
   - **Apple (AAPL)**: $213.55
   - **Tesla (TSLA)**: $315.35
   - **Other stocks (not bought due to strategy decisions)**: 
     - Google (GOOGL): $179.53
     - Microsoft (MSFT): $498.84
     - Amazon (AMZN): $223.41

3. **Transactions Executed**:
   - **Bought 30 shares of Apple (AAPL)** at an average price of $213.98 per share.
     - **Rationale**: Positive market sentiment in the tech sector suggests potential for growth.
   - **Bought 10 shares of Tesla (TSLA)** at an average price of $315.98 per share.
     - **Rationale**: Increasing demand for EVs bolsters Tesla's valuation.

### Portfolio Status:

- **Current Balance**: $420.88
- **Current Holdings**:
  - AAPL: 30 shares 
  - TSLA: 10 shares
- **Total Portfolio Value**: $9,980.88
- **Total Profit/Loss**: -$19.12

### Next Steps:
I will monitor the performance of these stocks closely, especially due to the active earnings season and market shifts. I will be ready to make further trades based on new information.

### Then go and look at the trace

http://platform.openai.com/traces


In [13]:
# And let's look at the results of the trading

await read_accounts_resource(agent_name)

'{"name": "ed", "balance": 420.87999999999965, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"AAPL": 30, "TSLA": 10}, "transactions": [{"symbol": "AAPL", "quantity": 30, "price": 213.9771, "timestamp": "2025-07-04 12:57:59", "rationale": "Positive market sentiment in tech sector and potential growth based on recent trends."}, {"symbol": "TSLA", "quantity": 10, "price": 315.9807, "timestamp": "2025-07-04 12:57:59", "rationale": "Tesla\'s valuation is strengthening as EV demand increases and it\'s at a competitive price point."}], "portfolio_value_time_series": [["2025-07-04 12:57:37", 10000.0], ["2025-07-04 12:57:38", 10000.0], ["2025-07-04 12:57:59", 9987.187], ["2025-07-04 12:58:00", 9980.88], ["2025-07-04 12:58:05", 9980.88]], "total_portfolio_value": 9980.88, "total_profit_loss": -19.120000000001255}'

### Now it's time to review the Python module made from this:

`mcp_params.py` is where the MCP servers are specified. You'll notice I've brought in some familiar friends: memory and push notifications!

`templates.py` is where the instructions and messages are set up (i.e. the System prompts and User prompts)

`traders.py` brings it all together.

You'll notice I've done something a bit fancy with code like this **async with AsyncExitStack() as stack**:

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

This is just a tidy way to combine our "with" statements (known as context managers) so that we don't need to do something ugly like this:

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

But it's equivalent.


In [14]:
from traders import Trader


In [15]:
trader = Trader("Ed")

In [16]:
await trader.run()

In [17]:
await read_accounts_resource("Ed")

'{"name": "ed", "balance": 1642.7744999999998, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"AAPL": 30, "TSLA": 5, "NIO": 100}, "transactions": [{"symbol": "AAPL", "quantity": 30, "price": 213.9771, "timestamp": "2025-07-04 12:57:59", "rationale": "Positive market sentiment in tech sector and potential growth based on recent trends."}, {"symbol": "TSLA", "quantity": 10, "price": 315.9807, "timestamp": "2025-07-04 12:57:59", "rationale": "Tesla\'s valuation is strengthening as EV demand increases and it\'s at a competitive price point."}, {"symbol": "NIO", "quantity": 100, "price": 3.5170199999999996, "timestamp": "2025-07-04 12:59:07", "rationale": "Strong year-over-year growth in deliveries and positive market sentiment in the EV sector."}, {"symbol": "TSLA", "quantity": -5, "price": 314.71930000000003, "timestamp": "2025-07-04 12:59:09", "rationale": "Taking partial profits due to current volatility and 

### Now look at the trace

https://platform.openai.com/traces

### How many tools did we use in total?

In [18]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("ed")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"We have {len(all_params)} MCP servers, and {count} tools")

We have 6 MCP servers, and 18 tools


In [19]:
for params in all_params:
    print(f"Params: {params['command']} {params['args']}")

Params: uv ['run', 'accounts_server.py']
Params: uv ['run', 'push_server.py']
Params: uv ['run', 'market_server.py']
Params: uvx ['mcp-server-fetch']
Params: npx ['-y', '@modelcontextprotocol/server-brave-search']
Params: npx ['-y', 'mcp-memory-libsql']


In [20]:
for tool in mcp_tools:
    print(tool)

name='create_entities' description='Create new entities with observations and optional embeddings' inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}, 'embedding': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Optional vector embedding for similarity search'}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']} annotations=None
name='search_nodes' description='Search for entities and their relations using text or vector similarity' inputSchema={'type': 'object', 'properties': {'query': {'oneOf': [{'type': 'string', 'description': 'Text search query'}, {'type': 'array', 'items': {'type': 'number'}, 'description': 'Vector for similarity search'}]}}, 'required': ['query']} annotations=None
name='read_graph' description='Get recent entities and their rel